<p style="font-size: 18px;">
  This is the accompanying code for the post titled "From Grid to Goal: A Practical Guide to A* Search."<br>
  You can find it <a href="https://pureai.substack.com/p/from-grid-to-goal-code-included">here</a>.<br>
  Published: August 5, 2023<br>
  <a href="https://pureai.substack.com">https://pureai.substack.com</a>
</p>

Welcome to this Jupyter notebook! If you're new to Python or don't have it installed on your system, don't worry; you can still follow along and explore the code.

Here's a quick guide to getting started:

- Using an Online Platform: You can run this notebook in a web browser using platforms like Google Colab or Binder. These services offer free access to Jupyter notebooks and don't require any installation.
- Installing Python Locally: If you'd prefer to run this notebook on your own machine, you'll need to install Python. A popular distribution for scientific computing is Anaconda, which includes Python, Jupyter, and other useful tools.
  - Download Anaconda from [here](https://www.anaconda.com/download).
  - Follow the installation instructions for your operating system.
  - Launch Jupyter Notebook from Anaconda Navigator or by typing jupyter notebook in your command line or terminal.
- Opening the Notebook: Once you have Jupyter running, navigate to the location of this notebook file (.ipynb) and click on it to open.
- Running the Code: You can run each cell in the notebook by selecting it and pressing Shift + Enter. Feel free to modify the code and experiment with it.
- Need More Help?: If you're new to Python or Jupyter notebooks, you might find these resources helpful:
  - [Python.org's Beginner's Guide](https://docs.python.org/3/tutorial/index.html)
  - [Jupyter Notebook Basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html)

Happy coding, and enjoy exploring the fascinating world of A* Search and State Space Search!

In [1]:
# A small readme
from platform import python_version
print(f"The version of Python used is: {python_version()}")

from typing import List, Tuple, Dict, Callable, Optional
import random
import heapq

################### Note ###################
# If you don't have heapq installed, you'll want to download the package from pip
# This can be done by running the following command in your terminal:
# pip install heapq or pip3 install heapq
################### Note ###################

The version of Python used is: 3.10.12


# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.

## Motivation

In video games, search techniques are commonly applied to path-finding. Even though game characters usually move within continuous spaces, it's straightforward to design a navigation system using a grid of "waypoints" overlaid on that space. To travel from Point A to Point B, a character first performs a line of sight (LOS) scan to identify the closest waypoint to its starting point (named Waypoint A) and the closest LOS waypoint to its destination (named Waypoint B). The A* search algorithm is then employed to determine the shortest path from Waypoint A to Waypoint B. Consequently, the complete journey comprises four segments: from Point A to Waypoint A, then to Waypoint B, and finally to Point B.

To make this concept more digestible, we'll represent the problem within a grid world. In this representation, different symbols on the grid will denote various types of terrain, each having its own cost for entry:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
🪨       hills      5
🐊       swamp      7
🗻       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾🗻🗻🗻🌾🌾🌾
🌾🌾🗻🗻🌾🌾🌾
🌾🌾🗻🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

# Graph Based State Space Search Pseudocode

In our quest to comprehend State Space Search, the following pseudocode serves as a vital guide for formulating our algorithm

```text
place initial state on frontier
initialize explored list
while frontier is not empty
	current-state := next state on frontier
	return path( current-state) if is-terminal( current-state)
	children := successors( current-state)
	for each child in children
		add child to frontier if not on explored or frontier
	add current-state to explored
return nil
```
This pseudocode outlines the basic framework for a State Space Search. Initially, it places the starting state on what's known as the "frontier" and initializes a list of explored states. As long as there are states on the frontier, the algorithm continues to explore.

Within each iteration, the algorithm:

- Selects the next state from the frontier (referred to as "current-state").
- Checks if this state is the goal (or a "terminal" state), and if so, returns the path to it.
- Identifies the children or successor states of the current state.
- Adds each child state to the frontier if it hasn’t been explored or already on the frontier.
- Finally, adds the current state to the explored list to prevent re-exploration.

If the frontier is emptied without finding a goal state, the algorithm returns nil, indicating no path found.

In the context of State Space Search, this pseudocode succinctly captures the essence of exploring a space systematically. It helps us not only to navigate through different states efficiently but also to ensure that we consider all possible paths to find the most optimal route to our goal.

### generate_world

Generate a random world with the given dimensions and the given number of obstacles.

* **m** Int: The number of rows in the world.
* **n** Int: The number of columns in the world.
* **start** Tuple(Int, Int): The starting position of the agent.
* **goal** Tuple(Int, Int): The goal position of the agent.

**returns** List[List[str]]: A randomly generated world of obstacles.

In [2]:
def generate_world(m: int, n: int, start: Tuple[int, int], goal: Tuple[int, int]) -> List[List[str]]:
    terrains = ['🌾', '🌲', '🪨', '🐊', '🗻']
    world = [[random.choice(terrains) for _ in range(n)] for _ in range(m)]
    # Ensure start and goal are passable
    world[start[0]][start[1]] = random.choice(terrains[:-1])  # Exclude '🗻'
    world[goal[0]][goal[1]] = random.choice(terrains[:-1])  # Exclude '🗻'
    return world

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [3]:
# Generate a large world with the starting point at (0,0) and the goal at (24,24)
large_world = generate_world(m=25, n=25, start=(0,0), goal=(24,24))

for row in large_world:
    print(''.join(row))

🐊🗻🗻🐊🗻🐊🌾🗻🐊🪨🌲🪨🗻🌾🐊🐊🌲🌲🪨🌲🪨🗻🌲🌲🗻
🌲🌾🌲🌲🪨🪨🐊🪨🌾🌲🌲🐊🗻🗻🪨🌾🌾🌾🌲🐊🗻🌲🌲🌾🐊
🗻🌾🗻🐊🐊🌾🪨🌲🌲🐊🌾🐊🗻🌾🌾🐊🐊🌲🪨🌾🗻🌲🌲🌲🌲
🪨🌲🌾🌲🐊🪨🌾🌾🗻🐊🌾🪨🌲🐊🐊🌲🌲🌲🪨🗻🗻🪨🗻🗻🌲
🌲🪨🐊🌲🌾🗻🌲🪨🐊🌾🌲🪨🌾🪨🌾🪨🪨🐊🪨🗻🐊🌲🐊🪨🪨
🪨🪨🌾🌾🌲🐊🗻🗻🐊🌲🌾🌾🐊🌾🪨🗻🌾🗻🗻🪨🪨🌲🗻🗻🗻
🐊🌾🌾🗻🪨🐊🪨🌾🐊🪨🗻🌾🗻🌲🗻🪨🌾🪨🗻🌲🌾🌲🌾🌾🗻
🌲🌾🌲🗻🐊🌾🪨🗻🗻🗻🪨🐊🌲🐊🌾🐊🐊🌲🗻🗻🌲🌲🪨🐊🗻
🗻🌾🗻🌲🌾🐊🪨🗻🐊🐊🐊🌾🪨🪨🪨🌾🌾🗻🪨🗻🐊🌲🗻🌲🐊
🪨🪨🗻🐊🐊🪨🗻🌾🪨🐊🗻🌲🪨🪨🌾🌲🌾🗻🗻🐊🌲🪨🌾🪨🌾
🐊🐊🌾🌲🌲🗻🗻🌲🪨🐊🌾🌲🪨🪨🌾🌲🌲🌾🌲🐊🌾🪨🪨🐊🌾
🌾🌲🪨🐊🐊🪨🌾🗻🪨🐊🌾🌲🐊🐊🌾🪨🐊🐊🌾🐊🌾🌲🌾🐊🌲
🌲🪨🐊🗻🗻🪨🗻🗻🌲🌾🌾🐊🐊🗻🌲🪨🐊🪨🌲🌾🐊🗻🐊🗻🌾
🌲🌾🪨🗻🌾🪨🌲🪨🌾🪨🪨🗻🌲🌾🪨🪨🪨🐊🐊🪨🌾🐊🗻🌲🗻
🐊🐊🪨🐊🌲🌾🪨🌾🪨🐊🗻🐊🗻🌲🌲🗻🌾🗻🐊🗻🌾🌲🗻🪨🪨
🐊🌾🪨🌲🌾🗻🌾🪨🪨🪨🗻🪨🪨🗻🌲🪨🪨🐊🌾🪨🗻🌾🌾🐊🪨
🌾🐊🪨🐊🌾🐊🐊🐊🗻🌾🗻🗻🌲🗻🪨🐊🌲🐊🌲🐊🗻🌾🗻🌲🌾
🌲🌾🌲🗻🌾🪨🪨🗻🌾🗻🗻🗻🗻🗻🐊🐊🌾🌲🌲🌾🌾🌾🌾🌲🗻
🌲🌲🗻🐊🪨🌲🗻🪨🪨🪨🌾🌲🐊🪨🐊🐊🌲🗻🌲🌾🪨🗻🗻🗻🌲
🌲🗻🐊🗻🌲🗻🌲🌲🪨🪨🪨🌾🗻🌲🗻🐊🐊🐊🐊🌾🌲🗻🗻🐊🗻
🌲🗻🪨🗻🌾🌾🗻🪨🌾🌲🌲🪨🌾🐊🐊🪨🌲🪨🌾🌾🗻🌾🗻🌲🌾
🐊🪨🐊🪨🗻🪨🪨🪨🌲🐊🪨🐊🪨🌲🪨🌲🐊🌲🪨🐊🗻🌲🗻🌾🪨
🌲🗻🌾🗻🪨🗻🐊🪨🌾🪨🗻🌲🪨🌲🌲🌾🗻🐊🌲🌾🌾🗻🌾🌲🐊
🪨🪨🐊🗻🐊🌾🪨🗻🪨🪨🌾🌲🪨🐊🪨🗻🌲🪨🌲🌲🐊🐊🐊🗻🌲
🌲🌲🗻🪨🪨🪨🪨🌲🐊🪨🗻🌲🌾🪨🗻🗻🗻🗻🪨🗻🐊🗻🪨🪨🌾


In [4]:
# Generate a large world with the starting point at (0,0) and the goal at (24,24)
small_world = generate_world(m=10, n=10, start=(0,0), goal=(9,9))

for row in small_world:
    print(''.join(row))

🌾🌾🌾🌾🌲🗻🌲🐊🌾🌲
🐊🗻🪨🌲🐊🗻🌲🌾🌾🌾
🌾🌲🐊🪨🐊🪨🗻🗻🪨🪨
🐊🪨🐊🌾🌲🗻🐊🌲🗻🌲
🐊🪨🌾🐊🌲🪨🗻🗻🐊🪨
🗻🐊🗻🌾🐊🌲🪨🪨🌲🌾
🌾🌲🌾🗻🌲🗻🌲🐊🌲🗻
🗻🗻🐊🌾🗻🗻🗻🌾🌲🌾
🌾🗻🌲🌾🐊🌾🌲🐊🌾🐊
🌲🪨🗻🌲🪨🐊🗻🌾🪨🌲


## Defining States and Their Representation

A State Space Search problem consists of four fundamental components: States, Actions, Transitions, and Costs.

In the context of our navigation problem, a state corresponds to the agent's current location, defined by coordinates (x,y). Rather than explicitly listing all possible states, they are implied within the framework of the world map.

## Actions and Transitions

We must now identify the actions that are possible within the world. These actions can encompass various movement directions such as north, south, east, west, or even diagonal directions. In essence, the agent’s potential movements, in combination with the established states, create the permissible actions that form the Transition set.

Instead of explicitly listing the Transition set, we can more efficiently determine the valid actions and transitions as needed. By employing a movement model that defines offsets to the current state, we can quickly check which among the possible successor states are allowed. This is managed within the successor function, which will be detailed in the accompanying pseudocode.

An example of this kind of movement model is illustrated below.

In [5]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [6]:
COSTS = { '🌾': 1, '🌲': 3, '🪨': 5, '🐊': 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [7]:

import unittest

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="heuristic"></a>
## heuristic

**The Manhattan distance calculation is used for the heuristic. This calculation is the total sum of the absolute value of the difference between two points. It should give a good approximation for the heuristic used in A* search.**

* **pos** Tuple[int, int]: the current position used to calculate the heuristic, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.

**returns** int: the heuristic calculated as the Manhattan distance between the pos and the goal coordinates as an `int`.

In [8]:
def heuristic(pos: Tuple[int, int], goal: Tuple[int, int]) -> int:
    # The Manhattan distance heuristic
    return abs(goal[0] - pos[0]) + abs(goal[1] - pos[1])

### Unit test for `heuristic`

In [9]:
test_cases = [
    ((0, 0), (0, 0), 0),
    ((0, 0), (1, 1), 2),
    ((0, 0), (-1, -1), 2),
    ((1, 2), (3, 4), 4),
    ((-1, -2), (-3, -4), 4),
    ((-1, 2), (3, -4), 10)
]

for pos, goal, expected in test_cases:
    assert heuristic(pos, goal) == expected, f"For {pos} and {goal}, expected {expected} but got {heuristic(pos, goal)}"

<a id="initialize_search"></a>
## initialize_search

**This function initializes the search by creating the initial states, the priority queue, and starting parent nodes.**

* **start** Tuple[int, int]: the starting position, `(x, y)`.
* **heuristic** Callable: the heuristic function used to calculate the heuristic for the A* search.
* **goal** Tuple[int, int]: the desired goal position, `(x, y)`.

**returns** Tuple[List[Tuple[int, Tuple[int, int]]], Dict[Tuple[int, int], int], Dict[Tuple[int, int], int], Dict[Tuple[int, int], int], Dict[Tuple[int, int], Optional[Tuple[int, int]]]]: the priority queue, initial states, and the starting parent nodes.

In [10]:
def initialize_search(start: Tuple[int, int], heuristic: Callable, goal: Tuple[int, int]) -> Tuple[List[Tuple[int, Tuple[int, int]]], Dict[Tuple[int, int], int], Dict[Tuple[int, int], int], Dict[Tuple[int, int], int], Dict[Tuple[int, int], Optional[Tuple[int, int]]]]:
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))

    G = {start: 0}  # The actual cost from the starting point to the current_pos position
    H = {start: heuristic(start, goal)}  # Heuristic cost of the current_pos position to the goal (this is a guess)
    F = {start: H[start]}  # Total estimated cost (F = G + H)
    parent_nodes = {start: None}
    
    return priority_queue, G, H, F, parent_nodes

### Unit Test for `initialize_search`

In [11]:
import math

# Define a start and goal
start = (0, 0)
goal = (5, 5)

# Initialize the search
priority_queue, G, H, F, parent_nodes = initialize_search(start, heuristic, goal)

# Assert the priority_queue contains the correct first element
assert priority_queue[0] == (0, start), f"Expected: [(0, {start})], but got: {priority_queue}"

# Assert the G, H, F, and parent_nodes dictionaries have been correctly initialized for the start node
assert G == {start: 0}, f"Expected: {{start: 0}}, but got: {G}"
assert math.isclose(H[start], heuristic(start, goal)), f"Expected: {heuristic(start, goal)}, but got: {H[start]}"
assert math.isclose(F[start], heuristic(start, goal)), f"Expected: {heuristic(start, goal)}, but got: {F[start]}"
assert parent_nodes == {start: None}, f"Expected: {{start: None}}, but got: {parent_nodes}"

print("All assertions passed!")

All assertions passed!


<a id="reconstruct_path"></a>
## reconstruct_path


**This function reconstructs the path from the start to the goal using the parent nodes.**

* **current_pos** Tuple[int, int]: the current position, `(x, y)`.
* **parent_nodes** Dict[Tuple[int, int], Optional[Tuple[int, int]]]: the parent nodes used to reconstruct the path.

**returns** List[Tuple[int, int]]: the path from the start to the goal.

In [12]:
def reconstruct_path(current_pos: Tuple[int, int], parent_nodes: Dict[Tuple[int, int], Tuple[int, int]]) -> List[Tuple[int, int]]:
    path = []
    while current_pos is not None:
        path.append(current_pos)
        current_pos = parent_nodes[current_pos]
    path.reverse()
    return path

### Unit Test for `reconstruct_path`

In [13]:
# Define parent_nodes dict that represents a hypothetical path search result
parent_nodes = {(0, 0): None, (1, 0): (0, 0), (2, 0): (1, 0), (2, 1): (2, 0), (3, 1): (2, 1), (4, 1): (3, 1)}

# Define the final position (the goal that was reached)
current_pos = (4, 1)

# Call the reconstruct_path function
path = reconstruct_path(current_pos, parent_nodes)

# Define the expected result
expected_path = [(0, 0), (1, 0), (2, 0), (2, 1), (3, 1), (4, 1)]

# Use an assert statement to check the reconstructed path matches the expected path
assert path == expected_path, f"Expected: {expected_path}, but got: {path}"

print("All assertions passed!")

All assertions passed!


<a id="explore_neighbors"></a>
## explore_neighbors

**This function explores the neighbors of the current position and adds them to the priority queue if they are valid.**

* **world** List[List[str]]: the world map.
* **current_pos** Tuple[int, int]: the current position, `(x, y)`.
* **moves** List[Tuple[int, int]]: the moves used to explore the neighbors.
* **costs** Dict[str, int]: the costs associated with each terrain type.
* **G** Dict[Tuple[int, int], int]: the G values for each position.
* **H** Dict[Tuple[int, int], int]: the H values for each position.
* **F** Dict[Tuple[int, int], int]: the F values for each position.
* **parent_nodes** Dict[Tuple[int, int], Optional[Tuple[int, int]]]: the parent nodes for each position.
* **priority_queue** List[Tuple[int, Tuple[int, int]]]: the priority queue used to explore the neighbors.
* **goal** Tuple[int, int]: the desired goal position, `(x, y)`.
* **heuristic** Callable: the heuristic function used to calculate the heuristic for the A* search.

**returns** the updated G, H, F, parent nodes, and priority queue.

In [14]:
def explore_neighbors(world: List[List[str]], current_pos: Tuple[int, int], moves: List[Tuple[int, int]], costs: Dict[str, int], G: Dict[Tuple[int, int], int], H: Dict[Tuple[int, int], int], F: Dict[Tuple[int, int], int], parent_nodes: Dict[Tuple[int, int], Tuple[int, int]], priority_queue: List[Tuple[int, int]], goal: Tuple[int, int], heuristic: Callable):
    for x, y in moves:
        neighbor_node = current_pos[0] + x, current_pos[1] + y
        if 0 <= neighbor_node[0] < len(world) and 0 <= neighbor_node[1] < len(world[0]) and world[neighbor_node[0]][neighbor_node[1]] in costs:
            tentative_g = G[current_pos] + costs[world[neighbor_node[0]][neighbor_node[1]]]

            if neighbor_node not in G or tentative_g < G[neighbor_node]:
                G[neighbor_node] = tentative_g
                H[neighbor_node] = heuristic(neighbor_node, goal)
                F[neighbor_node] = G[neighbor_node] + H[neighbor_node]
                parent_nodes[neighbor_node] = current_pos
                heapq.heappush(priority_queue, (F[neighbor_node], neighbor_node))

### Unit Test for `explore_neighbors`

In [15]:
# Define world, costs, moves, heuristic
world = [['.' for _ in range(5)] for _ in range(5)]
costs = {'.': 1}
moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up
heuristic = lambda pos, goal: abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])  # Manhattan distance

# Define initial position, goal, and call the initialization function
start = (0, 0)
goal = (2, 2)
priority_queue, G, H, F, parent_nodes = initialize_search(start, heuristic, goal)

# Call explore_neighbors function
explore_neighbors(world, start, moves, costs, G, H, F, parent_nodes, priority_queue, goal, heuristic)

# Define expected output
expected_G = {(0, 0): 0, (0, 1): 1, (1, 0): 1}
expected_parent_nodes = {(0, 0): None, (0, 1): (0, 0), (1, 0): (0, 0)}

# Check the output matches the expected output
assert G == expected_G, f"Expected: {expected_G}, but got: {G}"
assert parent_nodes == expected_parent_nodes, f"Expected: {expected_parent_nodes}, but got: {parent_nodes}"

print("All assertions passed!")

All assertions passed!


<a id="a_star_search"></a>
## a_star_search

*The A* search algorithm (a type of best-first search algorithm) uses both the cost and a heuristic to figure out the most optimal path for a robot to travel in the given world.*
*The goal of the algorithm is to minimize the total cost that is takes to travel from a starting point to a goal*

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.

In [16]:
def a_star_search(world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    priority_queue, G, H, F, parent_nodes = initialize_search(start, heuristic, goal)
    
    while priority_queue:
        _, current_pos = heapq.heappop(priority_queue)
        if current_pos == goal:
            return reconstruct_path(current_pos, parent_nodes)
        explore_neighbors(world, current_pos, moves, costs, G, H, F, parent_nodes, priority_queue, goal, heuristic)
    return []

### Unit test for `a_star_search`

In [17]:
test_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

expected_path = [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6), (6, 6)]
test_start = (0, 0)
test_goal = (len(test_world[0]) - 1, len(test_world) - 1)
test_path = a_star_search(test_world, test_start, test_goal, COSTS, MOVES, heuristic)
assert test_path == expected_path

print('All assertions passed!')

All assertions passed!


## pretty_print_path

*The `pretty_print_path` function computes the cost of the path that the `a_star_search` function returns.
As it computes the total cost, it replaces the path with symbols of arrows pointing in the directions of the allowed movement.
It then prints a new ASCII representation of the graph with the path symbols and returns the total path cost.*

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.

In [18]:
# Write a dictionary that maps the MOVES to the corresponding symbols
MOVE_SYMBOLS = { (0, -1): '⏪', (0, 1): '⏩', (-1, 0): '⏫', (1, 0): '⏬' }

In [19]:
def pretty_print_path(world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    # Copy the world so we don't modify the original.
    new_world = [list(row) for row in world]

    # Initialize total cost
    total_cost = 0

    # Replace symbols on the path.
    for i in range(1, len(path)):
        dx, dy = path[i][0] - path[i-1][0], path[i][1] - path[i-1][1]
        new_world[path[i-1][0]][path[i-1][1]] = MOVE_SYMBOLS[(dx, dy)]
        # Add cost to total cost
        total_cost += costs[world[path[i-1][0]][path[i-1][1]]]

    # Mark the goal with the present symbol
    new_world[goal[0]][goal[1]] = '🎁'

    # Print the new world with the path
    for row in new_world:
        print(''.join(row))

    return total_cost

### Unit test for `pretty_print_path`

In [20]:
expected_path_cost = 12
test_start = (0, 0)
test_goal = (len(test_world[0]) - 1, len(test_world) - 1)
test_path = a_star_search(test_world, test_start, test_goal, COSTS, MOVES, heuristic)
test_path_cost = pretty_print_path(test_world, test_path, test_start, test_goal, COSTS)
assert test_path_cost == expected_path_cost

print('All assertions passed!')

⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
All assertions passed!


## Journey 1. 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

In [21]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏩⏩⏩⏬🌲🗻🌲🐊🌾🌲
🐊🗻🪨⏬🐊🗻🌲🌾🌾🌾
🌾🌲🐊⏬🐊🪨🗻🗻🪨🪨
🐊🪨🐊⏩⏬🗻🐊🌲🗻🌲
🐊🪨🌾🐊⏩⏬🗻🗻🐊🪨
🗻🐊🗻🌾🐊⏩⏩⏩⏬🌾
🌾🌲🌾🗻🌲🗻🌲🐊⏬🗻
🗻🗻🐊🌾🗻🗻🗻🌾⏬🌾
🌾🗻🌲🌾🐊🌾🌲🐊⏬🐊
🌲🪨🗻🌲🪨🐊🗻🌾⏩🎁
total path cost: 52
[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4), (4, 5), (5, 5), (5, 6), (5, 7), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (9, 9)]


## Journey 2.

Execute `a_star_search` and `print_path` for the `large_world`

In [22]:
large_start = (0, 0)
large_goal = (len(large_world[0]) - 1, len(large_world) - 1)
large_path = a_star_search(large_world, large_start, large_goal, COSTS, MOVES, heuristic)
large_path_cost = pretty_print_path(large_world, large_path, large_start, large_goal, COSTS)
print(f"total path cost: {large_path_cost}")
print(large_path)

⏬🗻🗻🐊🗻🐊🌾🗻🐊🪨🌲🪨🗻🌾🐊🐊🌲🌲🪨🌲🪨🗻🌲🌲🗻
⏩⏬🌲🌲🪨🪨🐊🪨🌾🌲🌲🐊🗻🗻🪨🌾🌾🌾🌲🐊🗻🌲🌲🌾🐊
🗻⏬🗻🐊🐊🌾🪨🌲🌲🐊🌾🐊🗻🌾🌾🐊🐊🌲🪨🌾🗻🌲🌲🌲🌲
🪨⏩⏩⏩⏩⏩⏩⏬🗻🐊🌾🪨🌲🐊🐊🌲🌲🌲🪨🗻🗻🪨🗻🗻🌲
🌲🪨🐊🌲🌾🗻🌲⏩⏩⏩⏬🪨🌾🪨🌾🪨🪨🐊🪨🗻🐊🌲🐊🪨🪨
🪨🪨🌾🌾🌲🐊🗻🗻🐊🌲⏩⏩⏩⏬🪨🗻🌾🗻🗻🪨🪨🌲🗻🗻🗻
🐊🌾🌾🗻🪨🐊🪨🌾🐊🪨🗻🌾🗻⏬🗻🪨🌾🪨🗻🌲🌾🌲🌾🌾🗻
🌲🌾🌲🗻🐊🌾🪨🗻🗻🗻🪨🐊🌲⏩⏬🐊🐊🌲🗻🗻🌲🌲🪨🐊🗻
🗻🌾🗻🌲🌾🐊🪨🗻🐊🐊🐊🌾🪨🪨⏩⏩⏬🗻🪨🗻🐊🌲🗻🌲🐊
🪨🪨🗻🐊🐊🪨🗻🌾🪨🐊🗻🌲🪨🪨🌾🌲⏬🗻🗻🐊🌲🪨🌾🪨🌾
🐊🐊🌾🌲🌲🗻🗻🌲🪨🐊🌾🌲🪨🪨🌾🌲⏩⏩⏬🐊🌾🪨🪨🐊🌾
🌾🌲🪨🐊🐊🪨🌾🗻🪨🐊🌾🌲🐊🐊🌾🪨🐊🐊⏬🐊🌾🌲🌾🐊🌲
🌲🪨🐊🗻🗻🪨🗻🗻🌲🌾🌾🐊🐊🗻🌲🪨🐊🪨⏩⏬🐊🗻🐊🗻🌾
🌲🌾🪨🗻🌾🪨🌲🪨🌾🪨🪨🗻🌲🌾🪨🪨🪨🐊🐊⏩⏬🐊🗻🌲🗻
🐊🐊🪨🐊🌲🌾🪨🌾🪨🐊🗻🐊🗻🌲🌲🗻🌾🗻🐊🗻⏩⏬🗻🪨🪨
🐊🌾🪨🌲🌾🗻🌾🪨🪨🪨🗻🪨🪨🗻🌲🪨🪨🐊🌾🪨🗻⏬🌾🐊🪨
🌾🐊🪨🐊🌾🐊🐊🐊🗻🌾🗻🗻🌲🗻🪨🐊🌲🐊🌲🐊🗻⏬🗻🌲🌾
🌲🌾🌲🗻🌾🪨🪨🗻🌾🗻🗻🗻🗻🗻🐊🐊🌾🌲🌲⏬⏪⏪🌾🌲🗻
🌲🌲🗻🐊🪨🌲🗻🪨🪨🪨🌾🌲🐊🪨🐊🐊🌲🗻🌲⏬🪨🗻🗻🗻🌲
🌲🗻🐊🗻🌲🗻🌲🌲🪨🪨🪨🌾🗻🌲🗻🐊🐊🐊🐊⏬🌲🗻🗻🐊🗻
🌲🗻🪨🗻🌾🌾🗻🪨🌾🌲🌲🪨🌾🐊🐊🪨🌲🪨🌾⏬🗻🌾🗻🌲🌾
🐊🪨🐊🪨🗻🪨🪨🪨🌲🐊🪨🐊🪨🌲🪨🌲🐊🌲🪨⏬🗻🌲🗻🌾🪨
🌲🗻🌾🗻🪨🗻🐊🪨🌾🪨🗻🌲🪨🌲🌲🌾🗻🐊🌲⏩⏬🗻🌾🌲🐊
🪨🪨🐊🗻🐊🌾🪨🗻🪨🪨🌾🌲🪨🐊🪨🗻🌲🪨🌲🌲⏩⏩⏬🗻🌲
🌲🌲🗻🪨🪨🪨🪨🌲🐊🪨🗻🌲🌾🪨🗻🗻🗻🗻🪨🗻🐊🗻⏩⏩🎁
total path cost: 148
[(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 7), (4, 8), (4, 9), (4, 10), (5, 10), (5, 11), (5, 12), (5, 13), (6, 13), (7, 13), (7, 14), (8, 14), (8, 15), (8, 16), (9, 16), (10, 16), (10, 17), (10, 18), (11, 18), (12, 18), (12, 19), (13, 19), (13, 20), (14, 20), (14, 21), (15, 21)

##### Thank You for Exploring This Notebook!

I hope you found this journey through the code as engaging and insightful as I did in creating it. Your curiosity is what drives innovation, and I encourage you to take it a step further.

Feel free to play around with this code! Why not try changing the shape of the world or selecting new starting and goal points? Instead of the default starting point in the top left corner or the goal in the bottom right corner, you can experiment with different coordinates to see how the algorithm adapts.

Your creativity and experimentation not only enhance your understanding but also open new doors for exploration and innovation. Who knows? You might stumble upon a novel approach or insight that could contribute to the field.

If you have any thoughts, questions, or discoveries, please don't hesitate to reach out. Happy coding, and keep pushing the boundaries of what's possible! 